In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import boto3
import requests
from botocore import UNSIGNED
from botocore.config import Config
from IPython.display import Audio
from torchaudio.utils import download_asset
import torchaudio

In [4]:
audio_path = r"/public1/cjh/workspace/DepressionPrediction/dataset/EATD-Corpus/train/1/positive.wav"

In [5]:
waveform, sample_rate = torchaudio.load(audio_path)
waveform.shape

torch.Size([2, 296576])

In [6]:
sample_rate

16000

In [7]:
Audio(waveform.numpy(), rate=sample_rate)

In [11]:
import audonnx

model = audonnx.load("/public1/cjh/workspace/DepressionPrediction/model/component/emotion-path/model")
model

Input:
  signal:
    shape: [1, -1]
    dtype: tensor(float)
    transform: None
Output:
  hidden_states:
    shape: [1, 1024]
    dtype: tensor(float)
    labels: [hidden_states-0, hidden_states-1, hidden_states-2, (...), hidden_states-1021,
      hidden_states-1022, hidden_states-1023]
  logits:
    shape: [1, 3]
    dtype: tensor(float)
    labels: [arousal, dominance, valence]

In [12]:
type(model)

audonnx.core.model.Model

In [13]:
hidden_states = model(waveform.numpy(), sample_rate)['hidden_states']
hidden_states

array([[-0.00724801,  0.00541775, -0.00821333, ...,  0.00695953,
         0.00792024,  0.00543779]], dtype=float32)

In [20]:
hidden_states.shape

(1, 1024)

In [14]:
logits = model(waveform.numpy(), sample_rate)['logits']
logits

array([[0.27059796, 0.31354678, 0.4118928 ]], dtype=float32)

### try to convert wav2vec2.0 to native pytorch model
- 注意，此操作容易导致内存不够而引发问题。尤其是 16 GB内存的设备

In [16]:
import onnx2torch
from onnx2torch import convert
import onnx

In [17]:
model = onnx.load(r"/public1/cjh/workspace/DepressionPrediction/model/component/emotion-path/model/model.onnx")
type(model)

onnx.onnx_ml_pb2.ModelProto

In [18]:
pytorch_model = convert(model)
pytorch_model

GraphModule(
  (ReduceMean_0): OnnxReduceStaticAxes()
  (Sub_1): OnnxBinaryMathOperation()
  (Mul_2): OnnxBinaryMathOperation()
  (ReduceMean_3): OnnxReduceStaticAxes()
  (Sub_4): OnnxBinaryMathOperation()
  (Constant_5): OnnxConstant()
  (Add_6): OnnxBinaryMathOperation()
  (Sqrt_7): OnnxSqrt()
  (Div_8): OnnxBinaryMathOperation()
  (Unsqueeze_9): OnnxUnsqueezeStaticAxes()
  (Conv_10): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
  (Transpose_11): OnnxTranspose()
  (ReduceMean_12): OnnxReduceStaticAxes()
  (Sub_13): OnnxBinaryMathOperation()
  (Constant_14): OnnxConstant()
  (Pow_15): OnnxPow()
  (ReduceMean_16): OnnxReduceStaticAxes()
  (Constant_17): OnnxConstant()
  (Add_18): OnnxBinaryMathOperation()
  (Sqrt_19): OnnxSqrt()
  (Div_20): OnnxBinaryMathOperation()
  (initializers): Module()
  (Mul_21): OnnxBinaryMathOperation()
  (Add_22): OnnxBinaryMathOperation()
  (Transpose_23): OnnxTranspose()
  (Constant_24): OnnxConstant()
  (Div_25): OnnxBinaryMathOperation()
  (Erf_26): On

In [19]:
type(pytorch_model)

torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl

## i have forgot which way to choose.